# Linear Programming Problem Set

This notebook contains a collection of Linear Programming (LP) and Integer Linear Programming (ILP) modeling problems.  
Each section presents one problem description that can be later formulated and solved using optimization tools in Python.

In [ ]:
# installing and importing the mip package
!pip install mip

from mip import *

In [ ]:
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  if status != OptimizationStatus.OPTIMAL:
    return

  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
    if v.x > 0.001:
      print(f"{v.name} = {v.x:.2f}")

In [ ]:
def save(model, filename):
  model.write(filename)
  with open(filename, "r") as f:
    print(f.read())

## Section 1 - Steel Company (Transportation with Fixed Costs)
A steel company has 3 plants, each requiring a minimum monthly amount of ore. The company buys ore from 3 mines with maximum monthly production capacities.  
There is a fixed cost per mine plus a transport cost depending on the mine-plant pair.  
Formulate an optimization model to minimize the total purchase cost.

In [ ]:
n = 3

In [ ]:
model = Model(sense=MINIMIZE, solver_name=CBC)

x = [[model.add_var(var_type=INTEGER, lb=0, name = f"x_{i+1}_{j+1}") for j in range(n)] for i in range(n)]
y = [model.add_var(var_type=BINARY, name = f"y_{i+1}") for i in range(n)]

model.objective = 10*x[0][0] + 8*x[1][0] + 13*x[2][0] + 7*x[0][1] + 9*x[1][1] + \
                  + 16*x[2][1] + 6.5*x[0][2] + 10.8*x[1][2] + 12.6*x[2][2] + \
                  + 50000*y[0] + 40000*y[1] + 30000*y[2]

model += x[0][0] + x[1][0] + x[2][0] == 12300
model += x[0][1] + x[1][1] + x[2][1] == 15400
model += x[0][2] + x[1][2] + x[2][2] == 13300

model += x[0][0] + x[0][1] + x[0][2] <= 11500*y[0]
model += x[1][0] + x[1][1] + x[1][2] <= 16500*y[1]
model += x[2][0] + x[2][1] + x[2][2] <= 13000*y[2]

save(model, "model.lp")

In [ ]:
solve(model)

## Section 5 - Agricultural Planning
Three agricultural regions with different land and water availability can produce wheat, cotton, and soybeans.  
Each crop has water requirements, land constraints, and profit per unit.  
Formulate a linear program to allocate production for maximum profit.

In [ ]:
n = 3

In [ ]:
model = Model(sense=MAXIMIZE, solver_name=CBC)

x = [[model.add_var(var_type=INTEGER, lb=0, name = f"x_{i+1}_{j+1}") for j in range(n)] for i in range(n)]

model.objective = 400*(x[0][0] + x[0][1] + x[0][2]) + \
                  + 300*(x[1][0] + x[1][1] + x[1][2]) + \
                  + 100*(x[2][0] + x[2][1] + x[2][2])
                  
model += x[0][0] + x[1][0] + x[2][0] <= 400
model += x[0][1] + x[1][1] + x[2][1] <= 600
model += x[0][2] + x[1][2] + x[2][2] <= 300
  
model += x[0][0] + x[0][1] + x[0][2] <= 600
model += x[1][0] + x[1][1] + x[1][2] == 500
model += x[2][0] + x[2][1] + x[2][2] == 325

model += 3*x[0][0] + 2*x[1][0] + x[2][0] <= 600
model += 3*x[0][1] + 2*x[1][1] + x[2][1] <= 800
model += 3*x[0][2] + 2*x[1][2] + x[2][2] <= 375

save(model, "model.lp")

In [ ]:
solve(model)

## Section 6 - Cutting Stock (Steel Bars)
A workshop has 6-meter steel bars that must be cut into 2m, 3m, and 4m pieces in given quantities.  
Formulate an Integer Linear Program to minimize waste.

In [ ]:
n = 4

In [ ]:
model = Model(sense=MINIMIZE, solver_name=CBC)

x = [model.add_var(var_type=INTEGER, lb=0.0, name = f"x_{j+1}") for j in range(n)]

model.objective = xsum(x[j] for j in range(n))
                  
model += 3*x[0] + x[1] + x[2] >= 50
model += x[1] + 2*x[3] >= 60
model += x[2] >= 90

save(model, "model.lp")

In [ ]:
solve(model)

## Section 7 - Metal Strips (Cutting Problem)
A company must produce strips of given dimensions from larger sheets using specified cutting patterns.  
Formulate a model to minimize waste or material used.

In [ ]:
n = 4
m = 2

In [ ]:
model = Model(sense=MINIMIZE, solver_name=CBC)

x = [model.add_var(var_type=INTEGER, lb=0, name = f"x_{i+1}") for i in range(n)]
y = [model.add_var(var_type=INTEGER, lb=0, name = f"y_{i+1}") for i in range(m)]

model.objective = 30000*y[0] + 22000*y[1]

model += 2*x[0] + x[1] + 2*x[2] + 5*x[3] >= 2000
model += x[0] + x[1] + x[2] >= 1000
model += y[0] == (x[1] + x[3])/750
model += y[1] == (x[0] + x[2])/500

save(model, "model.lp")

In [ ]:
solve(model)

## Section 9 - Quality Control Inspectors
Two types of inspectors (different costs, speeds, and reliability) must inspect at least 1800 parts in 8 hours.  
There are limited numbers of each inspector type available.  
Formulate a model to minimize inspection costs (including error costs).

In [ ]:
n = 2

In [ ]:
model = Model(sense=MINIMIZE, solver_name=CBC)

x = [model.add_var(var_type=INTEGER, lb=0.0, name = f"x_{i+1}") for i in range(n)]

model.objective = 8*6.5*x[0] + 10*4.5*x[1]

model += 200*x[0] + 150*x[1] >= 1800
model += x[0] <= 8
model += x[1] <= 10

save(model, "model.lp")

In [ ]:
solve(model)

## Section 11 - Hospital Room Allocation
A hospital must decide how many 1-bed, 2-bed, and 3-bed rooms to build under constraints of maximum number of rooms, minimum number of beds, labor effort, and space.  
Formulate the problem for multiple objectives:  
- minimize labor,  
- maximize revenue,  
- maximize beds,  
- minimize construction space.

In [ ]:
n = 3

### Model 1

In [ ]:
model = Model(sense=MINIMIZE, solver_name=CBC)

x = [model.add_var(var_type=INTEGER, lb=0.0, name = f"x_{i+1}") for i in range(n)]

model.objective = 1*1*x[0] + 0.8*2*x[1] + 0.8*3*x[2]

model += x[0] + x[1] + x[2] <= 70
model += x[0] + 2*x[1] + 3*x[2] >= 120
model += 0.85*x[0] - 0.15*x[1] - 0.15*x[2] >= 0
model += 0.7*x[0] - 0.3*x[1] - 0.3*x[2] <= 0

save(model, "model.lp")

In [ ]:
solve(model)

### Model 2

In [ ]:
model = Model(sense=MAXIMIZE, solver_name=CBC)

x = [model.add_var(var_type=INTEGER, lb=0.0, name = f"x_{i+1}") for i in range(n)]

model.objective = 1*1*x[0] + 2*(1/2)*x[1] + 3*(1/3)*x[2]

model += x[0] + x[1] + x[2] <= 70
model += x[0] + 2*x[1] + 3*x[2] >= 120
model += 0.85*x[0] - 0.15*x[1] - 0.15*x[2] >= 0
model += 0.7*x[0] - 0.3*x[1] - 0.3*x[2] <= 0

save(model, "model.lp")

In [ ]:
solve(model)

### Model 3

In [ ]:
model = Model(sense=MAXIMIZE, solver_name=CBC)

x = [model.add_var(var_type=INTEGER, lb=0.0, name = f"x_{i+1}") for i in range(n)]

model.objective = x[0] + 2*x[1] + 3*x[2]

model += x[0] + x[1] + x[2] <= 70
model += x[0] + 2*x[1] + 3*x[2] >= 120
model += 0.85*x[0] - 0.15*x[1] - 0.15*x[2] >= 0
model += 0.7*x[0] - 0.3*x[1] - 0.3*x[2] <= 0

save(model, "model.lp")

In [ ]:
solve(model)